# 🚀 Multithreaded ResNet-18 Optimizer

**Maximum Performance Animal Classifier**

This notebook implements a highly optimized ResNet-18 model with:
- ⚡ **Multithreading** for data loading and processing
- 🔥 **Maximum CPU/GPU utilization**
- 🎯 **Best accuracy optimization techniques**
- 📊 **Phase 1 & Phase 2 submissions**

---

In [ ]:
# 🚀 Environment Detection & Multithreading Setup
import os
import multiprocessing as mp
import torch

# Environment Detection
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Google Colab detected")
except ImportError:
    IN_COLAB = False
    print("💻 Local Jupyter detected")

# Base path configuration (adjusted for Multithreaded_approach folder)
BASE_PATH = '/content' if IN_COLAB else '..'
print(f"📁 Base path: {BASE_PATH}")

# CPU/GPU Detection and Optimization
cpu_count = mp.cpu_count()
print(f"🖥️  Available CPU cores: {cpu_count}")

# Set optimal number of workers
NUM_WORKERS = min(cpu_count, 16)  # Cap at 16 to avoid memory issues
print(f"⚡ Using {NUM_WORKERS} workers for data loading")

# Torch optimization settings
torch.set_num_threads(cpu_count)
if torch.cuda.is_available():
    print(f"🔥 CUDA available - {torch.cuda.device_count()} GPUs")
    # Enable cudnn benchmarking for optimal performance
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("🍎 Metal Performance Shaders (MPS) available")
else:
    print("💪 Using optimized CPU with maximum threads")

In [ ]:
# 📦 Enhanced Package Installation with Performance Optimization
import sys
import subprocess
import warnings
warnings.filterwarnings('ignore')

def install_packages():
    # Core packages with performance optimizations
    pkgs = [
        'torch', 'torchvision', 'torchaudio',  # Latest PyTorch with optimizations
        'pandas', 'numpy', 'pillow', 'scikit-learn', 
        'tqdm', 'requests', 'matplotlib', 'seaborn',
        'albumentations',  # Advanced augmentations
        'timm',  # State-of-the-art models
        'tensorboard',  # Training visualization
    ]
    
    if IN_COLAB:
        pkgs.extend(['gdown', 'fastai'])  # Additional Colab optimizations
    
    print("🔧 Installing optimized packages...")
    for pkg in pkgs:
        try:
            subprocess.run([sys.executable, '-m', 'pip', 'install', pkg, '--upgrade'], 
                         check=True, capture_output=True)
            print(f"✅ {pkg}")
        except Exception as e:
            print(f"❌ Failed to install {pkg}: {e}")

install_packages()

# Set environment variables for maximum performance
os.environ['OMP_NUM_THREADS'] = str(cpu_count)
os.environ['MKL_NUM_THREADS'] = str(cpu_count)
print(f"🎯 Optimized for {cpu_count} CPU threads")

In [ ]:
# 🌐 Advanced Data Download & Organization
if IN_COLAB:
    import gdown
    print("📥 Downloading datasets in parallel...")
    
    # Parallel download using threading
    import threading
    
    def download_and_extract(url, filename, extract_to):
        try:
            gdown.download(url, f'{BASE_PATH}/{filename}', quiet=False)
            os.system(f'cd {BASE_PATH} && unzip -q {filename}')
            os.system(f'rm -rf {BASE_PATH}/__MACOSX')
            os.system(f'mv {BASE_PATH}/{extract_to}/* {BASE_PATH}/')
            os.system(f'rm -rf {BASE_PATH}/{extract_to} {BASE_PATH}/{filename}')
            print(f"✅ {filename} processed")
        except Exception as e:
            print(f"❌ Error with {filename}: {e}")
    
    # Download both datasets in parallel
    thread1 = threading.Thread(target=download_and_extract, 
                              args=('https://drive.google.com/uc?id=18MA0qKg1rqP92HApr_Fjck7Zo4Bwdqdu', 
                                   'HV-AI-2025.zip', 'HV-AI-2025'))
    thread2 = threading.Thread(target=download_and_extract,
                              args=('https://drive.google.com/uc?id=1aszVlQFQOwJTy9tt79s7x87VJyYw-Sxy',
                                   'HV-AI-2025-Test.zip', 'HV-AI-2025-Test'))
    
    thread1.start()
    thread2.start()
    thread1.join()
    thread2.join()
    
    print("🎉 All datasets downloaded and organized!")
else:
    print("💻 Assuming data is present in parent directory structure")
    print(f"   - Labeled data: {BASE_PATH}/labeled_data/")
    print(f"   - Unlabeled data: {BASE_PATH}/unlabeled_data/")
    print(f"   - Test images: {BASE_PATH}/test_images/")

In [ ]:
# 🔥 Enhanced Imports & Device Optimization
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import torch.nn.functional as F
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau
import torchvision.transforms as transforms
import torchvision.models as models

# Enhanced data science imports
import pandas as pd
import numpy as np
from PIL import Image, ImageFilter, ImageEnhance
import albumentations as A
from albumentations.pytorch import ToTensorV2
import timm

# Analysis and visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
import time
import gc

# Device optimization with detailed info
if torch.cuda.is_available():
    device = torch.device('cuda')
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f'🚀 Using CUDA - {gpu_name}')
    print(f'📊 GPU Memory: {gpu_memory:.1f} GB')
    # Enable mixed precision for faster training
    from torch.cuda.amp import autocast, GradScaler
    scaler = GradScaler()
    USE_AMP = True
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
    print('🍎 Using Metal Performance Shaders (MPS)')
    USE_AMP = False  # MPS doesn't support AMP yet
else:
    device = torch.device('cpu')
    print('💪 Using optimized CPU')
    USE_AMP = False

print(f"⚡ PyTorch version: {torch.__version__}")
print(f"🎯 Device: {device}")
print(f"🔥 Mixed Precision: {USE_AMP}")

In [ ]:
# 📊 Advanced Data Analysis & Preprocessing
print("🔍 Loading and analyzing dataset...")

# Load data with enhanced analysis
df = pd.read_csv(f'{BASE_PATH}/labeled_data/labeled_data.csv')
print(f"📈 Dataset shape: {df.shape}")
print(f"🏷️  Columns: {list(df.columns)}")

# Enhanced label encoding and analysis
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['label'])
num_classes = len(label_encoder.classes_)

print(f"\n🎯 Number of classes: {num_classes}")
print("📊 Class distribution:")
class_counts = df['label'].value_counts()
for label, count in class_counts.items():
    print(f"   {label}: {count} samples")

# Calculate class weights for balanced training
class_counts_array = np.bincount(df['encoded_label'])
class_weights = 1.0 / class_counts_array
class_weights = class_weights / class_weights.sum() * num_classes
class_weights_tensor = torch.FloatTensor(class_weights).to(device)

print(f"\n⚖️  Class weights calculated for balanced training")
print("📈 Data analysis complete!")

# Memory optimization
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()